In [25]:
# Importo librerías

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import Counter
import pprint
import sys
import spotipy.util as util

In [26]:
# Obtengo credenciales

client_credentials_manager = SpotifyClientCredentials("bcd05cdd915c49429590886e25342140", 
                                                      "0eae1090f5c347769d17b38d4892e303")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [27]:
# De la aplicación voy a recibir una lista de nombres de usuarios.
# Tomo la siguiente lista como ejemplo:

users = ["sedepa", "zltm3cqxt0u0s9k4e8oy564dk"]

In [5]:
# Para el paso 3 del layout de la app, voy a tener que devolver a cada usuario una lista con 
# el nombre de sus respectivas playlists, para que seleccione cuáles va a querer subir para 
# crear la lista conjunta.

# Por tanto, de la siguiente función voy a obtener un diccionario para devolver a la app, con
# el siguiente formato:

diccionario1 = {"user1": 
                   ["playlist1_name", "playlist2_name"],
                "user2": 
                   ["playlist1_name", "playlist2_name"]}


# Además, aunque al usuario en la app no se lo voy a mostrar, también voy a aprovechar para 
# obtener aquí el playlist_ID correspondiente a cada playlist, guardándolos en otro diccionario 
# con el siguiente formato:


diccionario2 = {"user1": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"},
                "user2": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"}}

In [28]:
# Defino la función que obtenga ambos diccionarios a partir de una lista de usuarios.

def get_user_playlists(users):
    
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    
    for user in users:
        playlists = sp.user_playlists(user)
        playlists_names = []
        users_playlists_names_IDs[user] = {}
        
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
            
        users_playlists_names[user] = playlists_names
        
    return users_playlists_names, users_playlists_names_IDs

In [29]:
# Guardo cada diccionario en una variable

users_playlists_names, users_playlists_names_IDs = get_user_playlists(users)

In [8]:
# El diccionario users_playlists_names va a ser devuelto a la app para que cada usuario
# seleccione de entre sus playlists cuáles quiere subir.

# Además, cada usuario va a poder filtrar las canciones de sus playlists seleccionadas 
# por los géneros que escoja.

# También van a tener que indicar cuál de ellos será el "host" (el usuario en cuyo perfil 
# se creará la lista conjunta), y el nombre de la lista conjunta. 

In [9]:
# Después de que los usuarios hayan seleccionado las playlists que quieren subir, el género
# de las canciones, el host y el nombre de la nueva lista, tengo que recibir 
# un diccionario con el siguiente formato:

mygroup = {"users": 
                   {"user1": {"playlists": [], "genres": []}, 
                    "user2": {"playlists": [], "genres": []}}, 
           "host": "user1",
           "new_playlist_name": "name"}


# Es posible que las listas de valores de "playlists" y "genres" incluyan solo
# el valor "All" o "Any", respectivamente.

# Si el valor de la lista de playlists es "All", tengo que seleccionar todas las playlists
# del perfil. 
# Si el valor de la lista de genres es "Any", tengo que seleccionar todas las canciones de
# las playlists que me pasan.

In [10]:
# Tomo el siguiente grupo como ejemplo:

mygroup = {"users": 
                   {"sedepa": {"playlists": ["MAD", "Correr", "A little of verguenza, please"],
                               "genres": ["spanish rock"]}, 
                    "zltm3cqxt0u0s9k4e8oy564dk": {"playlists": ["All"],
                                                  "genres": ["Any"]}}, 
           "host": "zltm3cqxt0u0s9k4e8oy564dk",
           "new_playlist_name": "ÉXITO"}


# El segundo usuario tiene 2 playlists en su perfil, por lo que el número total de playlists que
# debo unificar es 5. 
# Además, de las canciones de las 3 playlists del primer usuario, solo debo quedarme con las que
# tengan "spanish rock" como género.

In [11]:
# Defino una función que obtenga, para cada usuario, una lista con todas las canciones que hay 
# en todas las playlist que el usuario me ha pasado. Lo recopilo en un diccionario con este formato: 

# users_playlists_tracks = {user1: [{playlist1: [tracks]}, {playlist2: [tracks]}...], 
#                           user2: [{playlist1: [tracks]}, {playlist2: [tracks]}...]...}


def get_tracks_info(group):
    users_playlists_tracks = {}
    for user in group["users"].keys():
        tracks_info = []
        
        if group["users"][user]["playlists"] == ["All"]:
            for playlist_ID in users_playlists_names_IDs[user].values():
                tracks_info.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
        
        else:
            for playlist in group["users"][user]["playlists"]:
                tracks_info.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))
    
        users_playlists_tracks[user] = tracks_info
        
    return users_playlists_tracks

In [12]:
users_playlists_tracks = get_tracks_info(mygroup)

In [13]:
def get_final_tracks(users_playlists_tracks):
    
    final_tracks = {}
    
    final_track_names = []
    final_track_IDs = []
    final_track_artists = []

    for user in users_playlists_tracks:
        for playlist in users_playlists_tracks[user]:
            for track in playlist["items"]:
                artist_id = track["track"]["album"]["artists"][0]["external_urls"]["spotify"][32:]
                artist_info = sp.artist(artist_id)
                genre_in_artist = 0
                if mygroup["users"][user]["genres"] == ["Any"]:
                    final_track_names.append(track["track"]["name"])
                    final_track_IDs.append(track["track"]["id"])
                    final_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                else:
                    for genre in mygroup["users"][user]["genres"]:
                        if genre in artist_info["genres"]:
                            genre_in_artist += 1
                    if genre_in_artist > 0:
                        final_track_names.append(track["track"]["name"])
                        final_track_IDs.append(track["track"]["id"]) 
                        final_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                    
    
    for i in range(len(final_track_names)):
        final_tracks[final_track_IDs[i]] = [final_track_names[i], final_track_artists[i]]
        
    final_track_IDs = [item for items, c in Counter(final_track_IDs).most_common() for item in [items] * c]
    
    final_track_IDs_unique = []
    
    for track_ID in final_track_IDs:
        if track_ID not in final_track_IDs_unique:
            final_track_IDs_unique.append(track_ID)
    
    return final_tracks, final_track_IDs_unique, final_track_IDs

In [14]:
final_tracks, final_track_IDs_unique, final_track_IDs = get_final_tracks(users_playlists_tracks)

In [15]:
def create_dataframe(final_tracks, final_track_IDs_unique, final_track_IDs):

    tracks = []
    artists = []

    for ID in final_track_IDs:
        tracks.append(final_tracks.get(ID)[0])
        artists.append(final_tracks.get(ID)[1])

    final_tracks_df = pd.DataFrame(data = [final_track_IDs, tracks, artists],
                                   columns = range(len(final_track_IDs)), 
                                   index = ["ID", "Track", "Artist"]).T

    return final_tracks_df, final_track_IDs_unique

In [16]:
final_tracks_df, final_track_IDs_unique = create_dataframe(final_tracks, final_track_IDs_unique, final_track_IDs)

In [17]:
final_track_IDs_unique

['7CJGFIChxuwIAQb5GbqfRF',
 '0dWvKc40j9oBcmSpnQcZvg',
 '7rbsQlwPWCnz72GdzVOi9h',
 '4aAfLSx9IthpC3Pw5pNk3E',
 '2a2MWHc7gtuecUWU9WIasb',
 '0TIr4u2NmwrsffZfUZYnxA',
 '2huX4MIVEWQtzCpu6B05SV',
 '4nPDl3Dx9AYoaozt4TGwPC',
 '2ut1w3YL0Bd7FeKlnkc3bk',
 '5L6FXwF0sJGnIziz6fZg0A',
 '3Q8avvsQE7mtTXT2TrDx3I',
 '6z5HbBTccGlWGe6VuVmKHU',
 '0HeRFbjqrxHGrWKYXroCUx',
 '3PRmZkWJirNlQ1HsYfQlH2',
 '4tV5XLh3sPazTJuomEDmi3',
 '0aePOPP6t864hWQGL8izY5',
 '0JshnWfAOhnJiob29TL7XO',
 '07C9vwXsHulbAV27ocI8xE',
 '2cWq1IzhV9fvvM9QGjy2Wf',
 '5xl6BNDOgk9qcSV2eW5n6g',
 '1PxKcrXMab5yTwhrV3BxxN',
 '5IubU2pECgrJW5DBqOP0hd',
 '1x9V9ukzKTPB1jIwq9amHv',
 '1PpTQYSfbzTdQf7kk0J7OJ',
 '3kJ8Q5SLw5a2USQOST7hhe',
 '1bA2ZK7CFxEMnyn1dWP2jp',
 '0GSU6yLOJqO10ziQuBLWOE',
 '64dkTKKNLwbjBuU4xnS2ei',
 '3UkU7W1CNZJlYMh1Wn4FKh']

In [18]:
final_tracks_df

,ID,Track,Artist
0,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
1,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
2,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
3,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
4,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
5,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
6,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
7,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
8,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
9,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva


In [19]:
# Aquí puedo o pedirle al usuario que me de autorización para crear la lista en su perfil, 
# o crearla directamente en el perfil de Sponnect y pasarle el link al usuario en la app.

sponnect_username = "m11oiwplfr6aqk5svov48b40w"

token = util.prompt_for_user_token(sponnect_username,
                           "playlist-modify-public",
                           client_id = "bcd05cdd915c49429590886e25342140",
                           client_secret = "0eae1090f5c347769d17b38d4892e303",
                           redirect_uri = "https://example.com/callback/")

In [20]:
sp = spotipy.Spotify(auth=token)
final_playlist = sp.user_playlist_create(sponnect_username, mygroup["new_playlist_name"])

In [21]:
# Y finalmente, tengo que incluir las canciones de final_track_IDs_unique en final_playlist
                                             
sp.user_playlist_add_tracks(sponnect_username, final_playlist["uri"][17:], final_track_IDs_unique)

{'snapshot_id': 'Miw5OTUzYWE3MTk2NDBiNjEwZDQ4NzlhYTY1MGU3YTM2YTBjYjBkZjE0'}

In [24]:
# Para terminar, obtengo el url de la playlist:

final_playlist_url = final_playlist["external_urls"]["spotify"]